In [5]:
import pandas as pd
import numpy as np

np.random.seed(42) 

data =  pd.read_csv('public_light_antibody.csv') # public_cdrh3 public_cdr3l
data=data.sample(100)
data.to_csv('public_cdrl.csv',index=False)
data.head()

,cdrs,vgenes,label
1153,SSNIGSNY|RNN|AAWDDSLSGS,IGLV1-47,7
5114,NSNVGSHA|KNN|ATWDYSVNGPV,IGLV1-44,4
5655,SSDVGGYNY|EVN|TSYAGSSTYV,IGLV2-8,3
1323,SSNIGSNT|SNN|AAWDDSLNGDVV,IGLV1-44,26
4956,NIGSTS|DDS|RVWVRSSDRPV,IGLV3-21,4


In [1]:
import pandas as pd
import numpy as np

data =  pd.read_csv('p10_n1_h_df.csv')
data['label'] = 0
data['label'][data['count']!=1] = 1
print(len(data))
print(data.head())

824476
           cdr3     vgene  count  label
0     ARGHYGMDV   IGHV3-7    280      1
1  ARIGYSSSSFDY   IGHV3-7    255      1
2      ARSDWFDP  IGHV3-74    229      1
3     ARGHYGLDV  IGHV3-11    205      1
4     ARGGWSLDY  IGHV4-59    199      1


/home/user/tmp/ipykernel_28864/763140855.py:6: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  data['label'][data['count']!=1] = 1
/home/user/tmp/ipykernel_28864/763140855.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [2]:
pos_data = data[data['count']!=1]
print(len(pos_data))
neg_data = data[data['count']==1]
print(len(neg_data))

np.random.seed(42) 
shuffled_indices = np.random.permutation(len(pos_data))

train_size = int(0.8 * len(pos_data))
val_size = int(0.1 * len(pos_data))
test_size = len(pos_data) - train_size - val_size

train_indices = shuffled_indices[:train_size]
val_indices = shuffled_indices[train_size:train_size + val_size]
test_indices = shuffled_indices[train_size + val_size:]

train_pos = pos_data.iloc[train_indices].reset_index(drop=True)
val_pos = pos_data.iloc[val_indices].reset_index(drop=True)
test_pos = pos_data.iloc[test_indices].reset_index(drop=True)

def sample_neg(pos,neg,n):
    vgene_count = pos['vgene'].value_counts()
    match_df = []
    for vgene,count in vgene_count.items():
        match_data = neg[neg['vgene']==vgene]
        match_df.append(match_data.sample(n*count, random_state=42))
    neg_match = pd.concat(match_df)
    sampled_indices = neg_match.index
    neg_drop = neg.drop(sampled_indices)
    neg_match = neg_match.reset_index(drop=True)
    pos_neg = pd.concat([pos,neg_match], ignore_index=True)
    return neg_match,pos_neg,neg_drop

val_neg_match,val_pos_neg,val_neg_drop = sample_neg(val_pos,neg_data,1)
test_neg_match,test_pos_neg,test_neg_drop = sample_neg(test_pos,val_neg_drop,1)
train_pos_neg = pd.concat([train_pos,test_neg_drop], ignore_index=True)

print(len(val_pos_neg))
print(len(test_pos_neg))
print(len(train_pos_neg))
print(len(train_pos_neg)+len(test_pos_neg)+len(val_pos_neg))

412238
412238
82446
82450
659580
824476


In [11]:
print(test_pos_neg.head())
test_pos_neg[['cdr','vgene','label']].to_csv('public_cdrh3.csv',index=False)

   Unnamed: 0.1  Unnamed: 0                               cdr     vgene  \
0        146240      146240  GYTFTSYW|IDPSDSYT|ARYYYGSSYYAMDY  IGHV1-69   
1        108043      108043     GFTFSSYS|ISPSGGIT|AKDDAWGRYKH  IGHV3-23   
2        324428      324428   GGSVSRGGYY|TSYSEGT|AAADWESFYFDS  IGHV4-31   
3        233988      233988      RFTFSSYG|IRYDGSNK|ARGPSGYHNT  IGHV3-30   
4        248755      248755  GFTFSSSA|ISASGGST|AKDERWESPHYLDS  IGHV3-23   

   count  label  
0     13      1  
1     14      1  
2     10      1  
3     11      1  
4     11      1  


In [4]:
print(test_pos_neg.head())
test_pos_neg=test_pos_neg[['cdr3','vgene','label']].sample(1000)
test_pos_neg.to_csv('public_cdrh3.csv',index=False)

               cdr3     vgene  count  label
0         AKGGDYFDY   IGHV3-7     10      1
1        TRRTGTGFAY   IGHV6-6     12      1
2         ASGYDFFAY  IGHV1-26     22      1
3  ARDSLGAGTVGATSGY  IGHV4-39     15      1
4    ARDLGYSYGHPFDY  IGHV3-33     20      1


In [6]:
import pandas as pd
import numpy as np

data =  pd.read_csv('p5_n1_cdr3l_df.csv')
data['label'] = data['count']
print(len(data))
print(data.head())

172310
   Unnamed: 0.1  Unnamed: 0         cdr3     vgene  count  label
0             0           0    QAWDSSTVV   IGLV3-1    134    134
1             1           1  QSYDSSLSGSV  IGLV1-40    128    128
2             2           2   SSYTSSSTLV  IGLV2-14    124    124
3             3           3  GTWDSSLSAGV  IGLV1-51    124    124
4             4           4    QAWDSSTGV   IGLV3-1    123    123


In [7]:
pos_data = data[data['count']>=5]
print(len(pos_data))
neg_data = data[data['count']<5]
print(len(neg_data))
np.random.seed(42)
shuffled_indices = np.random.permutation(len(pos_data))

train_size = int(0.8 * len(pos_data))
val_size = int(0.1 * len(pos_data))
test_size = len(pos_data) - train_size - val_size

train_indices = shuffled_indices[:train_size]
val_indices = shuffled_indices[train_size:train_size + val_size]
test_indices = shuffled_indices[train_size + val_size:]

train_pos = pos_data.iloc[train_indices].reset_index(drop=True)
val_pos = pos_data.iloc[val_indices].reset_index(drop=True)
test_pos = pos_data.iloc[test_indices].reset_index(drop=True)

def sample_neg(pos,neg,n):
    vgene_count = pos['vgene'].value_counts()
    match_df = []
    for vgene,count in vgene_count.items():
        match_data = neg[neg['vgene']==vgene]
        match_df.append(match_data.sample(n*count, random_state=42))
    neg_match = pd.concat(match_df)
    sampled_indices = neg_match.index
    neg_drop = neg.drop(sampled_indices)
    neg_match = neg_match.reset_index(drop=True)
    pos_neg = pd.concat([pos,neg_match], ignore_index=True)
    return neg_match,pos_neg,neg_drop

val_neg_match,val_pos_neg,val_neg_drop = sample_neg(val_pos,neg_data,1)
test_neg_match,test_pos_neg,test_neg_drop = sample_neg(test_pos,val_neg_drop,1)
train_pos_neg = pd.concat([train_pos,test_neg_drop], ignore_index=True)

print(len(val_pos_neg))
print(len(test_pos_neg))
print(len(train_pos_neg))
print(len(train_pos_neg)+len(test_pos_neg)+len(val_pos_neg))

39367
132943
7872
7876
156562
172310


In [8]:
test_pos_neg[['cdr3','vgene','label']].to_csv('public_cdr3l.csv',index=False)